In [1]:
import numpy as np
import pandas as pd
import os
import h5py
import plotly.express as px
import plotly.graph_objects as go

from shutil import copy
import matplotlib.pyplot as plt

In [2]:
# return Lambert X and Y coordinates given a database

def lam_X(df):
    if (df.dy==0):
        df.dy=0.01
    if ( abs(df.dx) <= abs(df.dy) ):
        return np.sign(df.dy) * (2*(1 - df.dz))**0.5 * (2*np.arctan(df.dx/df.dy)/(np.pi**0.5))
    else:
        return np.sign(df.dx) * (2*(1 - df.dz))**0.5 * np.pi**0.5/2
    
def lam_Y(df):
    if ( abs(df.dx) <= abs(df.dy) ):
        return np.sign(df.dy) * (2*(1 - df.dz))**0.5 * np.pi**0.5/2
    else:
        return np.sign(df.dx) * (2*(1 - df.dz))**0.5 * (2*np.arctan(df.dy/df.dx)/(np.pi**0.5))    

In [44]:
def overwrite(mydata, Emin, Emax, EbinSize, zmax, zbinSize, detSize, MCpath, MCoutOG, MCoutNewName):
    # read my distributions and add to pandas dataframe
    el = pd.read_hdf(mydata, 'els')
    trsm_el = el.loc[el.outcome=='trsm'].copy()
    
    print ('fraction of transmitted electrons', len(trsm_el)/len(el))
    print ('total num el', len(trsm_el))
    
    # add escape distance column
    trsm_el['escp_dist'] = (z_max - trsm_el.last_z)/trsm_el.dz
    
    # add Lambert projections columns
    trsm_el['lam_X'] = trsm_el.apply(lam_X, axis=1)
    trsm_el['lam_Y'] = trsm_el.apply(lam_Y, axis=1)
    
    # make categories based on energy
    EbinEdges = int((Emax-Emin)/EbinSize) + 1
    enLabels = ["{0} - {1}".format(i, i+EbinSize-1) for i in range(Emin, Emax, EbinSize)]
    trsm_el['en_bins'] = pd.cut(trsm_el.final_E, bins=np.linspace(Emin, Emax, EbinEdges), right=False, labels=enLabels)
            
    # make categories based on depth
    zbinEdges = int(zmax/zbinSize) + 1
    zLabels = ["{0} - {1}".format(i, i+zbinSize-1) for i in range (0, zmax, zbinSize)]
    trsm_el['z_bins'] = pd.cut(trsm_el.escp_dist, bins=np.linspace(0, zmax, zbinEdges), right=False, labels=zLabels)
    
    L = (np.pi/2)**0.5

    # compute accum_e
    en_xedges = np.linspace(-L, L, detSize+1)
    en_yedges = np.linspace(-L, L, detSize+1)

    accum_e = np.zeros([detSize,detSize,EbinEdges-1])
    for binNum, enBin in enumerate(enLabels):
        # select only X and Y lambert proj values for this energy range
        selection =  trsm_el.loc[trsm_el.en_bins == enBin].copy()
        Xvals = selection.lam_X
        Yvals = selection.lam_Y
        hist2d,_,_ = np.histogram2d(Xvals.values, Yvals.values, bins=(en_xedges, en_yedges))
        accum_e[:,:,binNum] = hist2d
    
    # compute accum_z
    z_xedges = np.linspace(-L, L, 4)
    z_yedges = np.linspace(-L, L, 4)

    accum_z = np.zeros([int(detSize/7), int(detSize/7), zbinEdges-1, EbinEdges-1])
    for ebinNum, enBin in enumerate(enLabels):
        for zbinNum, zBin in enumerate(zLabels):
            # select only X and Y lambert proj values for this energy range
            selection = trsm_el.loc[(trsm_el.en_bins == enBin) & (trsm_el.z_bins == zBin)].copy()
            Xvals = selection.lam_X.values
            Yvals = selection.lam_Y.values
            hist2d,_,_ = np.histogram2d(Xvals, Yvals, bins=[z_xedges, z_yedges])
            accum_z[:,:,zbinNum, ebinNum] = hist2d
    print (accum_z[:,:, 1, 2], '\n')
    print (np.sum(accum_z[:,:,:,:], axis=(0, 1, 3)))
    print (zLabels, '\n')

    # copy MC out file locally
    if os.path.exists(MCoutOG):
        os.remove(MCoutOG)
    copy(MCpath+MCoutOG, MCoutOG)
    print ('copied', MCpath+MCoutOG, 'to', MCoutOG,'\n')
    
    # rename h5 file
    if os.path.exists(MCoutNewName):
        os.remove(MCoutNewName)
    os.rename(MCoutOG,  MCoutNewName) 
    print ('renamed', MCoutOG, 'to', MCoutNewName,'\n')
    
    # overwrite MCfoil
    with h5py.File(MCoutNewName, 'a') as file:
        del file['EMData/MCfoil/accum_e']
        file['EMData/MCfoil/accum_e'] = accum_e

        del file['EMData/MCfoil/accum_z']
        file['EMData/MCfoil/accum_z'] = accum_z

        del file['EMData/MCfoil/totnum_el']
        file['EMData/MCfoil/totnum_el'] = np.array(len(trsm_el))

        del file['EMData/MCfoil/numEbins']
        file['EMData/MCfoil/numEbins'] = EbinEdges-1

        del file['EMData/MCfoil/numzbins']
        file['EMData/MCfoil/numzbins'] = zbinEdges-1
        
        del file['NMLparameters/MCCLfoilNameList/depthmax']
        file['NMLparameters/MCCLfoilNameList/depthmax'] = zmax/10
        print ('max depth:', zmax/10, 'nm \n')
        
        del file['NMLparameters/MCCLfoilNameList/depthstep']
        file['NMLparameters/MCCLfoilNameList/depthstep'] = zbinSize/10
        print ('depth step:', zbinSize/10, 'nm \n')

In [5]:
Emin = 17000
Emax = 20000
EbinSize = 1000

detSize = 21

pathToMCout = '/home/elena/EMsoft/EMsoftPublic/EMsoftData/contrastInversion/'
MCoutOG = 'MCfoilout_700.h5'

# 1) z_max = 700 nm

In [6]:
z_max = 7000 # A

zbinSize = 1000 #A

## a) no diffraction

In [11]:
file_nodiff_700 = '/home/elena/Documents/Work/MC-discrete/data/TRSM_diff:False_thick:7000.0_mat:Si_mode:DS_elastic:Ruth_vanilla_tilt:0.0_Emin:17000.0_E0:20000.0_tolE:0.0001_tolW:1e-07.h5'
MCoutnew_nodiff = 'MCout_EP_700_nodiff.h5'

In [43]:
overwrite(mydata=file_nodiff_700, 
          Emin  =Emin, 
          Emax  =Emax, 
          EbinSize=EbinSize, 
          zmax  =z_max, 
          zbinSize=zbinSize, 
          detSize=detSize, 
          MCpath =pathToMCout,
          MCoutOG=MCoutOG, 
          MCoutNewName=MCoutnew_nodiff)

fraction of transmitted electrons 0.8110346203731418
total num el 8129
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]] 

[8.128e+03 1.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00]
['0 - 999', '1000 - 1999', '2000 - 2999', '3000 - 3999', '4000 - 4999', '5000 - 5999', '6000 - 6999'] 

copied /home/elena/EMsoft/EMsoftPublic/EMsoftData/contrastInversion/MCfoilout_700.h5 to MCfoilout_700.h5 

renamed MCfoilout_700.h5 to MCout_EP_700_nodiff.h5 

max depth: 700.0 nm /n
depth step: 100.0 nm /n


## b) with diffraction

In [30]:
file_wdiff_700 = '/home/elena/Documents/Work/MC-discrete/data/TRSM_diff:True_thick:7000.0_mat:Si_mode:DS_elastic:Ruth_vanilla_tilt:0.0_Emin:17000.0_E0:20000.0_tolE:0.0001_tolW:1e-07.h5'
MCoutnew = 'MCout_EP_700_wdiff.h5'

In [45]:
overwrite(mydata=file_wdiff_700, 
          Emin     = Emin, 
          Emax     = Emax, 
          EbinSize = EbinSize, 
          zmax     = z_max, 
          zbinSize = zbinSize, 
          detSize  = detSize, 
          MCpath   = pathToMCout,
          MCoutOG  = MCoutOG, 
          MCoutNewName=MCoutnew)

fraction of transmitted electrons 0.5063885006987423
total num el 5073
[[ 3.  2.  0.]
 [ 1. 59. 10.]
 [ 0.  5.  5.]] 

[4759.  141.   85.   41.   27.    7.    9.]
['0 - 999', '1000 - 1999', '2000 - 2999', '3000 - 3999', '4000 - 4999', '5000 - 5999', '6000 - 6999'] 

copied /home/elena/EMsoft/EMsoftPublic/EMsoftData/contrastInversion/MCfoilout_700.h5 to MCfoilout_700.h5 

renamed MCfoilout_700.h5 to MCout_EP_700_wdiff.h5 

max depth: 700.0 nm 

depth step: 100.0 nm 

